In [1]:
from numpy import array, zeros, ones
from numpy.linalg import inv, solve

## Input de Usuario

Para poder ingresar los datos al programa, se espera que el usuario
ingrese en un orden establecido la siguiente informacion para la
correcta definicion de los marcos a analizar.

123456789012345678901234567890123456789012345678901234567890123456789

## Definicion de Elementos

A continuacion se define cada uno de los parametros y sus respectivas
funciones para un objeto _Barra_ generico.

123456789012345678901234567890123456789012345678901234567890123456789

In [2]:
class barra():
    # user input at init
    # i j
    # x0 y0 x1 y1 # [cm]
    # b = 30 # [cm]
    # h = 40 # [cm]
    # ϰ = 1.2 # factor de forma de la seccion
    # 𝜈 = 0.25 # modulo de poisson
    # fc = 19.5 # [MPa]
    #
    # internals
    # L = ((x1 - x0)**2 + (y1 - y0)**2)**0.5 # [cm]
    # s = (y1 - y0)/L # seno
    # c = (x1 - x0)/L # coseno
    # A = b*h # [cm2]
    # I = b*h**3/12 # [cm4]
    # E = (15100*(fc*1000)**0.5/100)/0.01 # [kg/mm2]/0.01 = [kg/cm2]
    # G = E/(2*(1+𝜈)) # modulo de cizalle
    # β = 6*E*I*ϰ/(G*A*L**2)
    # k11 = 2*E*I*(2 + β)/(L*(1 + 2*β))
    # k12 = 2*E*I*(1 - β)/(L*(1 + 2*β))
    # k21 = k12
    # k22 = k11
    # k33 = A*E/L
    # k = array([
    #     [k11, k12, 0],
    #     [k21, k22, 0],
    #     [0,   0,   k33]
    # ]) # rigidez local
    # a = array([
    #     [-s/L, c/L, 1,  s/L, -c/L, 0],
    #     [-s/L, c/L, 0,  s/L, -c/L, 1],
    #     [c,    s,   0, -c,   -s,   0]
    # ]) # compatibilidad geometrica
    # K = a.T@k@a
    # σ = k@a@T@r_global

    def __init__(bar, nodos, i, j, x0, y0, x1, y1, b, h, ϰ, 𝜈, fc):
        #### ## input ## ####
        bar.GdL(i, j, nodos)
        bar.i = i # nodo A
        bar.j = j # nodo B
        bar.x0 = x0 # [cm]
        bar.y0 = y0 # [cm]
        bar.x1 = x1 # [cm]
        bar.y1 = y1 # [cm]
        bar.b = b # [cm]
        bar.h = h # [cm]
        bar.ϰ = ϰ # factor de forma de la seccion
        bar.𝜈 = 𝜈 # modulo de poisson
        bar.fc = fc # [MPa]

        #### ## internals ## ####
        bar.L = ((bar.x1 - bar.x0)**2 + (bar.y1 - bar.y0)**2)**0.5 # [cm]
        bar.s = (bar.y1 - bar.y0)/bar.L # seno
        bar.c = (bar.x1 - bar.x0)/bar.L # coseno
        bar.A = bar.b*bar.h # [cm2]
        bar.I = bar.b*bar.h**3/12 # [cm4]
        bar.E = (15100*(bar.fc*1000)**0.5/100)/0.01 # [kg/mm2]/0.01 = [kg/cm2]
        bar.G = bar.E/(2*(1+bar.𝜈)) # modulo de cizalle
        bar.β = 6*bar.E*bar.I*bar.ϰ/(bar.G*bar.A*bar.L**2)

        bar.k11 = 2*bar.E*bar.I*(2 + bar.β)/(bar.L*(1 + 2*bar.β))
        bar.k12 = 2*bar.E*bar.I*(1 - bar.β)/(bar.L*(1 + 2*bar.β))
        bar.k21 = bar.k12
        bar.k22 = bar.k11
        bar.k33 = bar.A*bar.E/bar.L

        bar.k = array([
            [bar.k11, bar.k12, 0      ],
            [bar.k21, bar.k22, 0      ],
            [0,       0,       bar.k33]
        ]) # rigidez local

        bar.a = array([
            [-bar.s/bar.L, bar.c/bar.L, 1,  bar.s/bar.L, -bar.c/bar.L, 0],
            [-bar.s/bar.L, bar.c/bar.L, 0,  bar.s/bar.L, -bar.c/bar.L, 1],
            [ bar.c,       bar.s,       0, -bar.c,       -bar.s,       0]
        ]) # compatibilidad geometrica

        bar.K = bar.a.T@bar.k@bar.a # rigidez global
        
    def GdL(bar, a, b, nodos, tipo=''):
        bar.T = zeros([6, 3*nodos])
        if a > nodos or b > nodos:
            print('index error')
            return
        if a < 0 or b < 0:
            print('index error')
            return
        if a != 0:
            bar.T[0, 3*a - 3] = 1
            bar.T[1, 3*a - 2] = 1
            bar.T[2, 3*a - 1] = 1
        if b != 0:
            bar.T[3, 3*b - 3] = 1
            bar.T[4, 3*b - 2] = 1
            bar.T[5, 3*b - 1] = 1

    def sigma(bar, r_global):
        # σ = k@ε
        # ε = a@r_i
        # r_i = T@r
        # σ = k@a@T@r
        bar.σ = bar.k@bar.a@bar.T@r_global # esfuerzos

    r = 0 # desplazamientos - sin uso
    ε = 0 # deformaciones - sin uso
    σ = 0 # esfuerzos

In [3]:
#__init__(nodos, i, j,  x0, y0, x1,  y1,  b,  h, ϰ,    𝜈,   fc)
col = barra(  5, 2, 5, 200, 0, 200, 300, 30, 40, 0, 0.25, 19.5)
print(col.T.T@ones((6,6))@col.T)

[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 1. 1. 0. 0. 0. 0. 0. 0. 1. 1. 1.]
 [0. 0. 0. 1. 1. 1. 0. 0. 0. 0. 0. 0. 1. 1. 1.]
 [0. 0. 0. 1. 1. 1. 0. 0. 0. 0. 0. 0. 1. 1. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 1. 1. 0. 0. 0. 0. 0. 0. 1. 1. 1.]
 [0. 0. 0. 1. 1. 1. 0. 0. 0. 0. 0. 0. 1. 1. 1.]
 [0. 0. 0. 1. 1. 1. 0. 0. 0. 0. 0. 0. 1. 1. 1.]]
